<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

Import e instalação das bibliotecas necessárias

In [ ]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

In [ ]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [ ]:
########## PATHS DE ARQUIVOS ##########

# Base completa bruta
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base teste bruta
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

In [ ]:
#Carrega a base de dados com stopWords
base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
base1.head()

In [ ]:
#Carrega a base de dados sem stopWords
base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
base2.head()

In [ ]:
base1.shape

In [ ]:
base2.shape

Classificação Hate, Hate+ e outros

In [ ]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

In [ ]:
# Parte de classificação hate
text = "lula tá na cadeia bolsonaro lula tá na cadeia a"
palavra_hate = 'lula tá na cadeia'
soma = 0
result = re.subn(r'^' + palavra_hate + '\s', palavra_hate + ' ',text) # Substitui no início
print(result[1])
soma+=result[1]
result = re.subn(r'\s' + palavra_hate + '$',' '+ palavra_hate,text) # Substitui no final
print(result[1])
soma+=result[1]
result = re.subn(r'\s' + palavra_hate + '\s', ' '+palavra_hate+ ' ',text) #Substitui no meio
print(result[1])
soma+=result[1]
print("soma:"+str(soma))